In [17]:
import os, sys, time, glob
import numpy as np
import scipy as sp
import mvpa2.suite as mvpa2

from scipy.spatial.distance import pdist, squareform

add_delay = 6
TR = 2.3

output_path = 'output'
offset = 5

brands = ['abercrombie-fitch','adobe','apple','axe','beats',
          'campina','dell','disney','durex','heineken','kelloggs',
          'microsoft','pampers','redbull']

cat_names = ['party','work','sex','family']

In [18]:
def ensure_dir(ed):
    try: 
        os.makedirs(ed)
    except OSError:
        if not os.path.isdir(ed):
            raise

In [19]:
def calculate_RSA(picture_response,brand_response):
    combined_sample = np.vstack([picture_response,brand_response])
    dist_matrix = squareform(pdist(combined_sample,metric='correlation'))
    return dist_matrix[len(picture_response):,:len(picture_response)]  

In [20]:
def calculate_SVM(picture_svm,brand_response):
    svm_result = {}
    for cn in cat_names:
        res = picture_svm[cn].predict(brand_response)
        prob = np.array([l[1][1] for l in picture_svm[cn].ca.probabilities])
        est = picture_svm[cn].ca.estimates
        svm_result[cn+'_prob'] = prob
        svm_result[cn+'_est'] = est
    return svm_result

In [62]:
method = 'normalized_common'
r = 'picture_thresh2'
method0 = method.split('_')[0]
method1 = method.split('_')[1]

Calculate scores from raw data

In [63]:
vpos = []
vpos.append(0)

for i,subj in enumerate(range(4,42)):
    if method1 == "spm":
        mask_file = '../roi/'+method0+'/common_mask/spm/picture2.nii'
    elif method1 == "common":
        if method0 == "native":
            mask_file = '../roi/'+method0+'/common_mask/'+r+'/S'+str(subj)+'/w'+r+'.nii'
        else:
            mask_file = '../roi/'+method0+'/common_mask/'+r+'.nii'
    else:
        mask_file = '../roi/'+method0+'/'+r+'/S'+str(subj)+'.nii'
        
    ds = mvpa2.fmri_dataset(mask_file,mask=mask_file,add_fa={'cluster':mask_file})
    vpos.append(vpos[-1]+ds.nfeatures)

print vpos

[0, 3173, 6346, 9519, 12692, 15865, 19038, 22211, 25384, 28557, 31730, 34903, 38076, 41249, 44422, 47595, 50768, 53941, 57114, 60287, 63460, 66633, 69806, 72979, 76152, 79325, 82498, 85671, 88844, 92017, 95190, 98363, 101536, 104709, 107882, 111055, 114228, 117401, 120574]


In [64]:
picture_ds = mvpa2.Dataset.from_hdf5('../output/'+method+'/'+method+'_'+r+'.h5py')
for cn in cat_names:
    picture_ds.sa[cn] = (picture_ds.sa.category == cn)

In [65]:
for n,subj in enumerate(range(4,42)):
    
    print 'Subject '+str(subj),
    t = time.time()

    #### Picture 
    
    pds = picture_ds[:,vpos[n]:vpos[n+1]]
    for x in range(len(pds)):
        pds.samples[x,:] = pds.samples[x,:] - np.mean(pds.samples[x,:])
    
    picture_svm = {}        
    for cn in cat_names:
        svm = mvpa2.LinearCSVMC(space=cn,probability=1,enable_ca=['probabilities'])
        svm.train(pds)
        picture_svm[cn] = svm

    picture_all_svm = mvpa2.LinearCSVMC(space='category')
    picture_all_svm.train(pds)
    
    picture_response = pds.samples
    
    #### Brand
    
    file_path = '../output/'+method+'/'+r.replace("picture","brand")
    brand_ds = mvpa2.Dataset.from_hdf5(file_path+'/S'+str(subj)+'.h5py')
    
    # timepoint (0)
    brand_response = np.copy(brand_ds.samples)
    for x in range(len(brand_response)):
        brand_response[x,:] = brand_response[x,:] - np.mean(brand_response[x,:])
    
    # - RSA
    res = calculate_RSA(picture_response,brand_response)
    brand_ds.sa['t0_rsa'] = res
        
    # - SVM
    res = calculate_SVM(picture_svm,brand_response)
    for cn in cat_names:
        brand_ds.sa['t0_'+cn+'_prob'] = res[cn+'_prob']
        brand_ds.sa['t0_'+cn+'_est'] = res[cn+'_est']
    res = picture_all_svm.predict(mvpa2.Dataset(brand_response))
    brand_ds.sa['t0_category'] = res
        
    # timepoint (0+1)
    brand_response = np.copy(brand_ds.samples)
    for i in range(len(brand_ds)):
        chunk = brand_ds[i].sa.chunks
        brand = brand_ds[i].sa.brand
        scan = brand_ds[i].sa.scan        
        mask = (brand_ds.sa.chunks == chunk) & (brand_ds.sa.brand == brand) & \
                (brand_ds.sa.scan == scan+1)
        if True in mask:
            brand_response[i,:] = brand_response[i,:] + brand_ds[mask].samples[0]
    brand_response = brand_response / 2
    for x in range(len(brand_response)):
        brand_response[x,:] = brand_response[x,:] - np.mean(brand_response[x,:])

    # - RSA
    res = calculate_RSA(picture_response,brand_response)
    brand_ds.sa['t01_rsa'] = res
        
    # - SVM
    res = calculate_SVM(picture_svm,brand_response)
    for cn in cat_names:
        brand_ds.sa['t01_'+cn+'_prob'] = res[cn+'_prob']
        brand_ds.sa['t01_'+cn+'_est'] = res[cn+'_est']
    res = picture_all_svm.predict(mvpa2.Dataset(brand_response))
    brand_ds.sa['t01_category'] = res
    
    # timepoint (0+1+2)
    brand_response = np.copy(brand_ds.samples)
    for i in range(len(brand_ds)):
        chunk = brand_ds[i].sa.chunks
        brand = brand_ds[i].sa.brand
        scan = brand_ds[i].sa.scan        
        mask = (brand_ds.sa.chunks == chunk) & (brand_ds.sa.brand == brand) & \
                (brand_ds.sa.scan == scan+1)
        if True in mask:
            brand_response[i,:] = brand_response[i,:] + brand_ds[mask].samples[0]
        mask = (brand_ds.sa.chunks == chunk) & (brand_ds.sa.brand == brand) & \
                (brand_ds.sa.scan == scan+2)
        if True in mask:
            brand_response[i,:] = brand_response[i,:] + brand_ds[mask].samples[0]
    brand_response = brand_response / 3
    for x in range(len(brand_response)):
        brand_response[x,:] = brand_response[x,:] - np.mean(brand_response[x,:])

    # - RSA
    res = calculate_RSA(picture_response,brand_response)
    brand_ds.sa['t012_rsa'] = res
        
    # - SVM
    res = calculate_SVM(picture_svm,brand_response)
    for cn in cat_names:
        brand_ds.sa['t012_'+cn+'_prob'] = res[cn+'_prob']
        brand_ds.sa['t012_'+cn+'_est'] = res[cn+'_est']
    res = picture_all_svm.predict(mvpa2.Dataset(brand_response))
    brand_ds.sa['t012_category'] = res
            
    # timepoint (0+1+2+3)
    brand_response = np.copy(brand_ds.samples)
    for i in range(len(brand_ds)):
        chunk = brand_ds[i].sa.chunks
        brand = brand_ds[i].sa.brand
        scan = brand_ds[i].sa.scan        
        mask = (brand_ds.sa.chunks == chunk) & (brand_ds.sa.brand == brand) & \
                (brand_ds.sa.scan == scan+1)
        if True in mask:
            brand_response[i,:] = brand_response[i,:] + brand_ds[mask].samples[0]
        mask = (brand_ds.sa.chunks == chunk) & (brand_ds.sa.brand == brand) & \
                (brand_ds.sa.scan == scan+2)
        if True in mask:
            brand_response[i,:] = brand_response[i,:] + brand_ds[mask].samples[0]
        mask = (brand_ds.sa.chunks == chunk) & (brand_ds.sa.brand == brand) & \
                (brand_ds.sa.scan == scan+3)
        if True in mask:
            brand_response[i,:] = brand_response[i,:] + brand_ds[mask].samples[0]

    brand_response = brand_response / 4
    for x in range(len(brand_response)):
        brand_response[x,:] = brand_response[x,:] - np.mean(brand_response[x,:])

    # - RSA
    res = calculate_RSA(picture_response,brand_response)
    brand_ds.sa['t0123_rsa'] = res
        
    # - SVM
    res = calculate_SVM(picture_svm,brand_response)
    for cn in cat_names:
        brand_ds.sa['t0123_'+cn+'_prob'] = res[cn+'_prob']
        brand_ds.sa['t0123_'+cn+'_est'] = res[cn+'_est']
    res = picture_all_svm.predict(mvpa2.Dataset(brand_response))
    brand_ds.sa['t0123_category'] = res
    
    mvpa2.save(brand_ds,file_path+'/S'+str(subj)+'.h5py')
    
    print "Elapsed time: "+str(time.time()-t)

Subject 4 Elapsed time: 32.4176449776
Subject 5 Elapsed time: 32.5513300896
Subject 6 Elapsed time: 33.1683239937
Subject 7 Elapsed time: 31.8701059818
Subject 8 Elapsed time: 34.2401919365
Subject 9 Elapsed time: 31.7747738361
Subject 10 Elapsed time: 32.5651590824
Subject 11 Elapsed time: 32.2101199627
Subject 12 Elapsed time: 30.6198179722
Subject 13 Elapsed time: 31.5479609966
Subject 14 Elapsed time: 31.773802042
Subject 15 Elapsed time: 32.8431229591
Subject 16 Elapsed time: 32.1192359924
Subject 17 Elapsed time: 32.4548099041
Subject 18 Elapsed time: 32.3071389198
Subject 19 Elapsed time: 31.8924629688
Subject 20 Elapsed time: 32.2759521008
Subject 21 Elapsed time: 31.69333601
Subject 22 Elapsed time: 32.6601510048
Subject 23 Elapsed time: 31.5477080345
Subject 24 Elapsed time: 31.4628300667
Subject 25 Elapsed time: 31.2700431347
Subject 26 Elapsed time: 31.5625669956
Subject 27 Elapsed time: 30.6373441219
Subject 28 Elapsed time: 31.3012800217
Subject 29 Elapsed time: 31.608677

Calculate scores from beta images

In [55]:
f = open('../nuisance/brand_brightness.txt')
brand_brightness = []
for bb in f.readlines()[:14]:
    brand_brightness.append(np.log(float(bb)))
f.close()
f = open('../nuisance/picture_brightness.txt')
picture_brightness = []
for pb in f.readlines()[:112]:
    picture_brightness.append(np.log(float(pb)))
f.close()

In [61]:
if method1 == "common":
    if method0 == "native":
        mask_file = '../roi/'+method0+'/common_mask/'+r+'/S'+str(subj)+'/w'+r+'.nii'
    else:
        mask_file = '../roi/'+method0+'/common_mask/'+r+'.nii'
else:
    mask_file = '../roi/'+method0+'/'+r+'/S'+str(subj)+'.nii'

brand_beta = 'entire_beta'
picture_beta = 'T3' # ~3TRs

for n,subj in enumerate(range(4,42)):
    
    print 'Subject '+str(subj),
    t = time.time()

    #### Picture 
    
    file_path = '../output/'+method+'/'+r.replace("picture","brand")+"_"+brand_beta
    ensure_dir(file_path)
    
    if brand_beta == 'logo_beta':
        brand_files = glob.glob('../spm/normalized/brand/S'+str(subj)+'/byphase/beta*.nii')[:14]        
    #elif brand_beta == 'cue_beta':
    #    brand_files = glob.glob('../spm/normalized/brand/S'+str(subj)+'/byphase/beta*.nii')[14*6:28*6]
    elif brand_beta == 'imagery_beta':
        #brand_files = glob.glob('../spm/normalized/brand/S'+str(subj)+'/byphase/beta*.nii')[28*6:42*6]
        brand_files = glob.glob('../spm/normalized/brand/S'+str(subj)+'/byphase/beta*.nii')[14:28]
    elif brand_beta == 'entire_beta':
        brand_files = glob.glob('../spm/normalized/brand/S'+str(subj)+'/T4O0/beta*.nii')[:14]
    
    brand_ds = mvpa2.fmri_dataset(brand_files,mask=mask_file)
    nan_mask = ~np.any(np.isnan(brand_ds.samples),axis=0)

    brand_ds = brand_ds[:,nan_mask]
    
    ###
    brand_ds.sa['brightness'] = brand_brightness #np.tile(brand_brightness,6)
    opt_regs = ['brightness']   
    mvpa2.poly_detrend(brand_ds,polyord=0,opt_regs=opt_regs)
    ###
    
    print np.sum(~nan_mask),
    brand_response = np.copy(brand_ds.samples)  
    
    pds = mvpa2.fmri_dataset(glob.glob('../spm/normalized/picture/S'+str(subj)+'/'+picture_beta+'/beta*.nii')[:112],mask=mask_file)
    pds = pds[:,nan_mask]
    
    ###
    pds.sa['brightness'] = picture_brightness
    opt_regs = ['brightness']   
    mvpa2.poly_detrend(pds,polyord=0,opt_regs=opt_regs)
    ###
    
    log_file = '../log/S'+str(subj)+'_picture.csv'
    log = np.recfromcsv(log_file,delimiter=',')
    pds.sa['category'] = sorted(log.category)
    for cn in cat_names:
        pds.sa[cn] = (pds.sa.category == cn)
        
    for x in range(len(pds)):
        pds.samples[x,:] = pds.samples[x,:] - np.mean(pds.samples[x,:])
        
    picture_svm = {}        
    for cn in cat_names:
        svm = mvpa2.LinearCSVMC(space=cn,probability=1,enable_ca=['probabilities'])
        svm.train(pds)
        picture_svm[cn] = svm

    picture_all_svm = mvpa2.LinearCSVMC(space='category')
    picture_all_svm.train(pds)
    
    picture_response = pds.samples
    
    #### Brand    
    
    for x in range(len(brand_response)):
        brand_response[x,:] = brand_response[x,:] - np.mean(brand_response[x,:])
    
    # - RSA
    res = calculate_RSA(picture_response,brand_response)
    brand_ds.sa['t0_rsa'] = res
        
    # - SVM
    res = calculate_SVM(picture_svm,brand_response)
    for cn in cat_names:
        brand_ds.sa['t0_'+cn+'_prob'] = res[cn+'_prob']
        brand_ds.sa['t0_'+cn+'_est'] = res[cn+'_est']
    res = picture_all_svm.predict(mvpa2.Dataset(brand_response))
    brand_ds.sa['t0_category'] = res
    
    brand_ds.sa['timepoint'] = [0 for i in range(len(brand_ds))]
    brand_ds.sa['brand'] = np.array(brands)
    #brand_ds.sa['brand'] = np.tile(brands,6)
    #brand_ds.sa['session'] = np.repeat(range(1,7),14)
    
    mvpa2.save(brand_ds,file_path+'/S'+str(subj)+'.h5py')
    
    print "Elapsed time: "+str(time.time()-t)

Subject 4 1 Elapsed time: 2.44670987129
Subject 5 0 Elapsed time: 2.59818005562
Subject 6 1 Elapsed time: 2.43993496895
Subject 7 0 Elapsed time: 2.86993503571
Subject 8 1 Elapsed time: 2.60699510574
Subject 9 3 Elapsed time: 2.56259894371
Subject 10 0 Elapsed time: 2.45740604401
Subject 11 0 Elapsed time: 2.36713790894
Subject 12 0 Elapsed time: 2.45674204826
Subject 13 5 Elapsed time: 2.63715291023
Subject 14 2 Elapsed time: 2.52398896217
Subject 15 2 Elapsed time: 2.29017090797
Subject 16 0 Elapsed time: 2.43927097321
Subject 17 0 Elapsed time: 2.5789899826
Subject 18 6 Elapsed time: 2.5779709816
Subject 19 0 Elapsed time: 2.65700387955
Subject 20 1 Elapsed time: 2.69838500023
Subject 21 0 Elapsed time: 2.4872469902
Subject 22 3 Elapsed time: 2.43124294281
Subject 23 0 Elapsed time: 2.4371111393
Subject 24 2 Elapsed time: 2.5626950264
Subject 25 0 Elapsed time: 2.5580508709
Subject 26 1 Elapsed time: 2.44645500183
Subject 27 6 Elapsed time: 2.56282091141
Subject 28 0 Elapsed time: 2